In [1]:
import os
import sys
import pandas as pd
import numpy as np

from DAWG.DAWG import dawg
from utils.utils import get_project_root
from automatos.conversion.conversor import conversor
from sklearn.metrics import *

<h1>1. Introdução</h1>


<h2>1.1 Conjunto de dados</h2>

<p>Para este trabalho foram utilizados dois conjuntos de dados, um para o
   treino(construção do dawg) e outra para as inferencias(teste), os conjuntos
   foram:</p>

   - **waltz.txt** para construção do dawg
   - **waltzdb.csv** para as inferências gramaticais

<p>Vale ressaltar que o conjunto de teste, contem dados de sequencias, na qual cada uma contem uma
   classificação, podendo ser amyloid ou non-amyloid.</p>

<h2>1.2 leitura dos dados</h2>

In [2]:
train_path = get_project_root() + '/Automatos-Bioinformatica/datasets/' + 'waltz.txt'
train = pd.read_csv(train_path, header=None)

test_path = get_project_root() + '/Automatos-Bioinformatica/datasets/' + 'waltzdb.csv'
test = pd.read_csv(test_path)

<h2>1.3 Divisão do treino em palavras positivas e negativas</h2>

<p>No conjunto de treino, existem dois tipos de sequencia(string), as positivas e as negativas
    de acordo com o algoritmo de criação de dawg em
<a href="http://proceedings.mlr.press/v34/wieczorek14a.pdf">wieczorek14a</a>, logo é necessario dividir o conjunto
 de treino em strings positivas e outras.</p>

In [3]:
plus_words = []
negative_words = []
for i in range(len(train)):
    if '\t+' in train.iloc[i][0]:
        w = train.iloc[i][0]
        plus_words.append(w[:len(w) - len('\t+')])
    else:
        negative_words.append(train.iloc[i][0])

<h1>2. Desenvolvimento do dawg com não determinismo</h1>

In [4]:
print("Treino")
dawg_alg = dawg()
dawg_alg.create(plus_words, negative_words)

Treino


<h1>3. Inferência das seguências com dawg não deterministico</h1>

In [5]:
target = 'amyloid'
N, M = test.shape

expected_number_target = len(test[test['Classification'] == target])
expected_number_non_target = N - expected_number_target
result = {
    'amyloid': 0,
    'non-amyloid': 0
}
y_out = []
for i in range(N):
    seguence = test.iloc[i]['Sequence']
    c = test.iloc[i]['Classification']

    seguence += '+'
    token = dawg_alg.in_automato_language(seguence)
    if token:
        if c == target:
            result[target] += 1
        y_out.append(target)
    elif not token:
        if c != target:
            result['non-amyloid'] += 1
        y_out.append('non-amyloid')
print(result)
print(expected_number_target)
print(expected_number_non_target)

{'amyloid': 121, 'non-amyloid': 811}
244
845


<h2>3.1 Calculando acurácia, precisão e recall</h2>

In [6]:
y_test = test['Classification'].to_numpy()

y_out = np.array(y_out)
print('ACC:', str(accuracy_score(y_out, y_test)))
print('R:', str(recall_score(y_out, y_test, average='binary', pos_label="amyloid")))
print('P:', str(precision_score(y_out, y_test, average='binary', pos_label="amyloid")))


ACC: 0.8558310376492194
R: 0.7806451612903226
P: 0.4959016393442623


In [16]:
conv = conversor()
conv._to_deterministic(dawg_alg)

AttributeError: 'AtlasView' object has no attribute 'update'